In [2]:
pwd

'/home/wsuser/work'

# Image Processing

###### Importing Image Data Generator Library



In [3]:
from keras.preprocessing.image import ImageDataGenerator

###### Configure Image datagenerator class



In [4]:
#setting parameter for image data agumentationto the training data
train_datagen=ImageDataGenerator(rescale=1./225,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#image data agumenatation to test dataset
test_datagen=ImageDataGenerator(rescale=1./225)

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_4WuTrixJC-84ULdX6SpGq8nRsETtMSQD_CjAKOseUb3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'classificationofarrythmia-donotdelete-pr-cww9lpnv74wol0'
object_key = 'Classification of Arrhythmia by Using Deep Learning with 2-D ECG Spectral Image Representation.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

###### Applying ImageDataGenerator Functionality to train and test Dataset



In [7]:
pwd

'/home/wsuser/work'

In [8]:
#performing data agumentation to train the dataset
x_train=train_datagen.flow_from_directory(directory='/home/wsuser/work/data/train',target_size=(64,64),batch_size=32,class_mode='categorical')
#performing agumentation to test the dataset
x_test=test_datagen.flow_from_directory(directory='/home/wsuser/work/data/test',target_size=(64,64),batch_size=32,class_mode='categorical')


Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


# Model Building

###### Importing Libraries



In [9]:
import numpy as np
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from tensorflow.keras.layers import Dense,Flatten

from tensorflow.keras.layers import Conv2D,MaxPooling2D
import keras

In [10]:
model= keras.Sequential()

###### Adding CNN Layers



In [11]:
#adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())

###### Adding Dense Layers



In [12]:
model.add(Dense(32))
model.add(Dense(6,activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 32)                2

###### Configuring the learning process



In [13]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


###### Train the Model



In [14]:
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))

/tmp/wsuser/ipykernel_164/53529210.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=x_train,steps_per_epoch = len(x_train), epochs=10, validation_data=x_test,validation_steps = len(x_test))


Epoch 1/10
480/480 [==============================] - 62s 127ms/step - loss: 0.8179 - accuracy: 0.7227 - val_loss: 0.5469 - val_accuracy: 0.8086
Epoch 2/10
480/480 [==============================] - 60s 124ms/step - loss: 0.3020 - accuracy: 0.9089 - val_loss: 0.4820 - val_accuracy: 0.8590
Epoch 3/10
480/480 [==============================] - 59s 123ms/step - loss: 0.2491 - accuracy: 0.9282 - val_loss: 0.5169 - val_accuracy: 0.8577
Epoch 4/10
480/480 [==============================] - 60s 124ms/step - loss: 0.2144 - accuracy: 0.9366 - val_loss: 0.4201 - val_accuracy: 0.8779
Epoch 5/10
480/480 [==============================] - 59s 123ms/step - loss: 0.1911 - accuracy: 0.9447 - val_loss: 0.3932 - val_accuracy: 0.8891
Epoch 6/10
480/480 [==============================] - 59s 123ms/step - loss: 0.1777 - accuracy: 0.9467 - val_loss: 0.2952 - val_accuracy: 0.9190
Epoch 7/10
480/480 [==============================] - 61s 127ms/step - loss: 0.1608 - accuracy: 0.9510 - val_loss: 0.3841 - val_ac

###### Saving the model



In [15]:
model.save('ECG_IBM.h5')

In [16]:
!tar -zcvf ECG-arrhythmia-classification-model_new.tgz ECG_IBM.h5

ECG_IBM.h5


In [17]:
ls -1

data/
ECG-arrhythmia-classification-model_new.tgz
ECG_IBM.h5


In [18]:
!pip install watson-machine-learning-client --upgrade


     |████████████████████████████████| 538 kB 20.6 MB/s eta 0:00:01


In [19]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"rrkTJwpDakBrJE-RdG1N2M2wnDmDQj7Qvl-TnlziN3Gy"
}
client=APIClient(wml_credentials)

In [20]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    #print(space)
    return(next(item for item in space ['resources']if item['entity']["name"]==space_name)['metadata']['id'])

In [21]:
space_uid=guid_from_space_name(client,'Classification of Arrhythmia')
print("Space UID= "+space_uid)

Space UID= 7277f3b7-fd81-4943-a294-8bca90dab1f6


In [22]:
client.set.default_space(space_uid)


'SUCCESS'

In [23]:
client.software_specifications.list()


-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [24]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [25]:
model_details = client.repository.store_model(model='ECG-arrhythmia-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [26]:
model_id

'674dcec5-ca13-495d-b9a5-2a5af1e68e9d'

In [27]:
client.repository.download(model_id,'my_model.tar1.gz')

Successfully saved model content to file: 'my_model.tar1.gz'


'/home/wsuser/work/my_model.tar1.gz'

###### Test the Model



In [33]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model=load_model("ECG_IBM.h5")


In [34]:
img = image.load_img("/home/wsuser/work/data/test/Premature Atrial Contraction/fig_26.png",target_size=(64,64))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)
pred = model.predict(x)
y_pred = np.argmax(pred)
y_pred

2

In [35]:
index=['left Bundle Branch block','Normal','Premature Atrial Contraction','Premature Ventricular Contraction','Right Bundle Branch Block','Ventricular Fibrillation']
result = str(index[y_pred])
result

'Premature Atrial Contraction'